In [1]:
import os
os.environ["NAPARI_ASYNC"] = "1"
os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [2]:
from copy import deepcopy
def get_spec(ndims):
    return {
        "driver": "zarr3",
        "kvstore": {
            "driver": "file",
            "path": None
        },
    "metadata": {
        "shape": None,
        "chunk_grid": {"name": "regular", "configuration": {"chunk_shape": ([1]*(ndims-2))+[32768, 32768]}},
        "chunk_key_encoding": {"name": "default"},
        "codecs": [{"name" : "sharding_indexed",
                    "configuration":{
                    "chunk_shape": ([1]*(ndims-2))+[512, 512],
                    "codecs":[{"name": "blosc", "configuration": {"cname": "lz4", "clevel": 5}}],
                    }}],
        "data_type": "uint32",
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':'open',
    }

def get_read_spec(filename):
    read_spec = deepcopy(get_spec(3))
    read_spec["kvstore"]["path"] = str(filename)
    del read_spec["metadata"]["shape"]
    return read_spec

def get_write_spec(filename, shape):
    write_spec = deepcopy(get_spec(3))
    write_spec["create"] = True
    write_spec["delete_existing"] = True
    write_spec["kvstore"]["path"] = str(filename)
    write_spec["metadata"]["shape"] = list(shape)
    return write_spec

In [3]:
import numpy as np
import tensorstore as ts
import tensorstore_trackarr as tta
import json

labels = np.load("original_labels.npy")
write_spec = get_write_spec("labels.zarr", labels.shape)
ts.open(write_spec).result().write(labels).result()

read_spec = get_read_spec("labels.zarr")
labels = ts.open(read_spec).result()

def convert_keys_and_values_to_int(d):
    return {int(key): [int(v) for v in value] for key, value in d.items()}

with open("original_split_dict.json", "r") as f:
    split_dict = json.load(f, object_hook=convert_keys_and_values_to_int)
print(split_dict)
ta = tta.TrackArray(labels, split_dict, {})

image = np.load("image.npy")

{7: [10, 9], 8: [12, 11], 9: [17, 16]}


In [4]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
viewer.add_image(image, translate=[0,1000,1000])
labels_layer = viewer.add_labels(ta.array, translate=[0,1000,1000])
labels_layer.editable=False
widget = StateMachineWidget(viewer, labels_layer, ta)
viewer.window.add_dock_widget(widget, area="right")

ERROR:root:Something went wrong while patching
Traceback (most recent call last):
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/utils/perf/_patcher.py", line 194, in patch_callables
    _patch_attribute(module, attribute_str, patch_func)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/utils/perf/_patcher.py", line 44, in _patch_attribute
    raise PatchError(
    ...<5 lines>...
    )
napari.utils.perf._patcher.PatchError: Nested attribute not found: chunk._loader.ChunkLoader.load_chunk
ERROR:root:Something went wrong while patching
Traceback (most recent call last):
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/utils/perf/_patcher.py", line 194, in patch_callables
    _patch_attribute(module, attribute_str, patch_func)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/fukai/projects/napa

AttributeError: 'Labels' object has no attribute 'array'

In [4]:
ta.splits

{7: [10, 9], 8: [12, 11], 9: [17, 16], 1: [20, 21]}

In [2]:
from transitions import Machine
from qtpy.QtWidgets import QPushButton, QVBoxLayout, QWidget, QLabel
from qtpy.QtCore import Qt
from napari import Viewer

class StateMachineWidget(QWidget):
    def __init__(self, viewer: Viewer):
        super().__init__()

        self.viewer = viewer

        # Set up the layout
        self.layout = QVBoxLayout()

        # Add a label to show the current state
        self.state_label = QLabel("Current State: idle")
        self.layout.addWidget(self.state_label)

        # Add buttons to trigger transitions
        self.start_button = QPushButton("Start")
        self.pause_button = QPushButton("Pause")
        self.reset_button = QPushButton("Reset")

        self.layout.addWidget(self.start_button)
        self.layout.addWidget(self.pause_button)
        self.layout.addWidget(self.reset_button)

        self.setLayout(self.layout)

        # Define the state machine
        self.states = ["idle", "running", "paused"]
        self.machine = Machine(model=self, states=self.states, initial="idle")

        # Add transitions
        self.machine.add_transition("start", "idle", "running")
        self.machine.add_transition("pause", "running", "paused")
        self.machine.add_transition("resume", "paused", "running")
        self.machine.add_transition("reset", ["paused", "running"], "idle")

        # Connect buttons to state transitions
        self.start_button.clicked.connect(self.on_start)
        self.pause_button.clicked.connect(self.on_pause)
        self.reset_button.clicked.connect(self.on_reset)

        # Bind viewer key events to transitions
        self.viewer.bind_key("s", lambda _viewer: self.on_start(), overwrite=True)  # Press 'S' to start
        self.viewer.bind_key("p", lambda _viewer: self.on_pause(), overwrite=True)  # Press 'P' to pause/resume
        self.viewer.bind_key("r", lambda _viewer: self.on_reset(), overwrite=True)  # Press 'R' to reset

    def on_start(self):
        if self.state == "idle":
            self.start()
            self.update_label()

    def on_pause(self):
        if self.state == "running":
            self.pause()
            self.update_label()
        elif self.state == "paused":
            self.resume()
            self.update_label()

    def on_reset(self):
        if self.state in ["running", "paused"]:
            self.reset()
            self.update_label()

    def update_label(self):
        self.state_label.setText(f"Current State: {self.state}")

viewer = Viewer()
widget = StateMachineWidget(viewer)
viewer.window.add_dock_widget(widget, area="right")

Traceback (most recent call last):
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/app_model/backends/qt/_qaction.py", line 59, in _on_triggered
    self._app.commands.execute_command(self._command_id).result()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/app_model/registries/_commands_reg.py", line 247, in execute_command
    raise e
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/app_model/registries/_commands_reg.py", line 242, in execute_command
    future.set_result(cmd(*args, **kwargs))
                      ~~~^^^^^^^^^^^^^^^^^
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/in_n_out/_store.py", line 934, in _exec
    result = func(*args, **kwargs)
  File "/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/in_n_out/_store.py", line 804, in _exec
    result = func(**bou

In [ ]:
@log_error
def track_clicked(layer, event):
    logger.info("Track clicked")
    yield  # important to avoid a potential bug when selecting the daughter
    logger.info("button released")
    data_coordinates = layer.world_to_data(event.position)
    logger.debug(f"world coordinates: {event.position}")
    logger.debug(f"data coordinates: {data_coordinates}")
    cords = np.round(data_coordinates).astype(int)
    val = layer.get_value(data_coordinates)
    try:
        _ = iter(val)
        val = val[-1]
    except TypeError as te:
        pass
        
    if val is None:
        return
    if val != 0:
        frame = cords[0]
        logger.info(f"clicked at {cords} at frame {frame} and label value {val}")
        self.track_clicked(frame, val, cords)
self._cropped_label_layer.mouse_drag_callbacks.append(track_clicked)